# 데이터 정제 및 가공

이 노트북은 1단계에서 통합된 데이터를 정제하고 가공하는 프로세스를 담고 있습니다.

## 프로젝트 개요
- **목적**: 통합된 데이터의 품질 개선 및 분석 준비
- **데이터 소스**: `../data/processed/01_consolidated_data.csv`
- **주요 작업**: 
  - 데이터 불러오기 및 기본 정보 확인
  - 결측값 처리
  - 중복 데이터 제거
  - 데이터 타입 최적화
  - 이상값 탐지 및 처리
  - 최종 정제된 데이터 저장


## 1. 필요한 라이브러리 불러오기

데이터 정제에 필요한 라이브러리들을 import합니다.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정 (맥OS)
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False


## 2. 데이터 불러오기

1단계에서 저장한 통합 데이터를 불러옵니다.


In [2]:
# 1단계에서 저장한 통합 데이터 불러오기
file_path = '../data/processed/01_consolidated_data.csv'

try:
    df = pd.read_csv(file_path)
    print("데이터 불러오기 성공!")
    print(f"데이터 크기: {df.shape[0]}개의 행, {df.shape[1]}개의 열")
    display(df.head())
except FileNotFoundError:
    print(f"[오류] 파일을 찾을 수 없습니다: {file_path}")
    print("1단계 노트북 마지막에 파일이 정상적으로 저장되었는지 확인해주세요.")


데이터 불러오기 성공!
데이터 크기: 36113개의 행, 67개의 열


,mb_sn,Q1,Q2,Q3,Q4,Q5,Q5_1,Q6,Q7,Q8,...,문항1_qpoll_join_250714,문항1_qpoll_join_250221,문항1_qpoll_join_250626,문항1_qpoll_join_250723,문항1_qpoll_join_250627,문항1_qpoll_join_250328,문항1_qpoll_join_250703,문항1_qpoll_join_250611,문항1_qpoll_join_250304,문항2_qpoll_join_250304
0,w131784019041454,2.0,3.0,5.0,1.0,5,13,4.0,4.0,"1,2,3,4,5,,7,8,9,,11,,,14,15,,17,18,,20,,22,23...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,w10403443472424,2.0,1.0,3.0,2.0,4,1,8.0,8.0,"1,2,3,4,5,,7,,9,,11,12,,,15,16,,,19,20,,22,23,...",...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,w181629889821619,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,w223949436321039,2.0,2.0,4.0,1.0,8,21,5.0,5.0,"1,2,,4,5,6,,,9,,11,12,,,15,,,,,,,22,23,,25,,,28",...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,4.0
4,w303750980346998,1.0,NaN,1.0,3.0,4,1,3.0,3.0,"1,2,,4,5,,,,9,,,,,,,,,18,,20,,22,23,,25,26,,",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 3. 데이터 기본 정보 확인

데이터의 구조와 품질을 파악합니다.


In [3]:
# 데이터 기본 정보
print("=== 데이터 기본 정보 ===")
print(f"데이터 크기: {df.shape}")
print(f"메모리 사용량: {df.memory_usage(deep=True).sum() / 1024**2:.1f}MB")

print("\n=== 데이터 타입 ===")
print(df.dtypes.value_counts())

print("\n=== 결측값 현황 ===")
missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100
missing_df = pd.DataFrame({
    '결측값 개수': missing_data,
    '결측값 비율(%)': missing_percent
})
missing_df = missing_df[missing_df['결측값 개수'] > 0].sort_values('결측값 개수', ascending=False)
print(missing_df.head(10))


=== 데이터 기본 정보 ===
데이터 크기: (36113, 67)
메모리 사용량: 59.9MB

=== 데이터 타입 ===
float64    37
object     30
Name: count, dtype: int64

=== 결측값 현황 ===
                       결측값 개수  결측값 비율(%)
Q12_2_ETC               36019  99.739706
Q13_ETC                 35813  99.169274
Q12_1_ETC               35630  98.662531
문항1_qpoll_join_250723   31582  87.453272
문항1_qpoll_join_250714   31557  87.384045
문항1_qpoll_join_250716   31552  87.370199
문항1_qpoll_join_250710   31546  87.353585
문항1_qpoll_join_250709   31541  87.339739
문항1_qpoll_join_250707   31536  87.325894
문항1_qpoll_join_250704   31493  87.206823


## 4. 중복 데이터 확인 및 제거

중복된 데이터가 있는지 확인하고 제거합니다.


In [4]:
# 중복 데이터 확인
print("=== 중복 데이터 분석 ===")
print(f"전체 행 수: {len(df)}")
print(f"고유한 mb_sn 개수: {df['mb_sn'].nunique()}")
print(f"중복된 mb_sn 개수: {len(df) - df['mb_sn'].nunique()}")

# 중복된 mb_sn 확인
duplicated_sn = df[df.duplicated(subset=['mb_sn'], keep=False)]
print(f"\n중복된 mb_sn이 있는 행 수: {len(duplicated_sn)}")

if len(duplicated_sn) > 0:
    print("\n중복된 mb_sn 예시:")
    print(duplicated_sn[['mb_sn']].head(10))
    
    # 중복 제거 (첫 번째 값만 유지)
    print("\n중복 제거 중...")
    df_clean = df.drop_duplicates(subset=['mb_sn'], keep='first')
    print(f"중복 제거 후 행 수: {len(df_clean)}")
    
    # 원본 데이터 업데이트
    df = df_clean
    print(f"데이터가 중복 제거된 데이터로 업데이트되었습니다.")
else:
    print("중복된 데이터가 없습니다.")


=== 중복 데이터 분석 ===
전체 행 수: 36113
고유한 mb_sn 개수: 36113
중복된 mb_sn 개수: 0

중복된 mb_sn이 있는 행 수: 0
중복된 데이터가 없습니다.


## 5. 결측값 처리

결측값을 적절히 처리합니다.


In [5]:
# 결측값 처리 전 상태
print("=== 결측값 처리 전 ===")
print(f"전체 결측값 개수: {df.isnull().sum().sum()}")
print(f"결측값이 있는 컬럼 수: {(df.isnull().sum() > 0).sum()}")

# 결측값 처리 전략
print("\n=== 결측값 처리 전략 ===")
print("1. 숫자형 컬럼: 0 또는 평균값으로 대체")
print("2. 문자형 컬럼: 'Unknown' 또는 'N/A'로 대체")
print("3. 설문 응답 컬럼: 결측값 유지 (응답하지 않음)")

# 숫자형 컬럼의 결측값을 0으로 대체
numeric_columns = df.select_dtypes(include=[np.number]).columns
df[numeric_columns] = df[numeric_columns].fillna(0)

# 문자형 컬럼의 결측값을 'Unknown'으로 대체
string_columns = df.select_dtypes(include=['object']).columns
df[string_columns] = df[string_columns].fillna('Unknown')

print("\n=== 결측값 처리 후 ===")
print(f"전체 결측값 개수: {df.isnull().sum().sum()}")
print(f"결측값이 있는 컬럼 수: {(df.isnull().sum() > 0).sum()}")


=== 결측값 처리 전 ===
전체 결측값 개수: 1597329
결측값이 있는 컬럼 수: 66

=== 결측값 처리 전략 ===
1. 숫자형 컬럼: 0 또는 평균값으로 대체
2. 문자형 컬럼: 'Unknown' 또는 'N/A'로 대체
3. 설문 응답 컬럼: 결측값 유지 (응답하지 않음)

=== 결측값 처리 후 ===
전체 결측값 개수: 0
결측값이 있는 컬럼 수: 0


## 6. 최종 데이터 확인 및 저장

정제된 데이터의 최종 상태를 확인하고 저장합니다.


In [6]:
# 최종 데이터 확인
print("=== 최종 데이터 확인 ===")
print(f"데이터 크기: {df.shape[0]}개의 행, {df.shape[1]}개의 열")
print(f"메모리 사용량: {df.memory_usage(deep=True).sum() / 1024**2:.1f}MB")
print(f"결측값 개수: {df.isnull().sum().sum()}")

# 데이터 정보
print("\n=== 데이터 정보 ===")
df.info()

# 데이터 미리보기
print("\n=== 데이터 미리보기 ===")
display(df.head())

# 정제된 데이터 저장
output_path = '../data/processed/02_cleaned_data.csv'
df.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"\n정제된 데이터가 '{output_path}'에 저장되었습니다.")
print(f"저장된 파일 크기: {df.shape}")

print("\n🎉 데이터 정제 완료!")
print("이제 분석을 진행할 수 있습니다.")


=== 최종 데이터 확인 ===
데이터 크기: 36113개의 행, 67개의 열
메모리 사용량: 77.9MB
결측값 개수: 0

=== 데이터 정보 ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36113 entries, 0 to 36112
Data columns (total 67 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   mb_sn                  36113 non-null  object 
 1   Q1                     36113 non-null  float64
 2   Q2                     36113 non-null  float64
 3   Q3                     36113 non-null  float64
 4   Q4                     36113 non-null  float64
 5   Q5                     36113 non-null  object 
 6   Q5_1                   36113 non-null  object 
 7   Q6                     36113 non-null  float64
 8   Q7                     36113 non-null  float64
 9   Q8                     36113 non-null  object 
 10  Q9_1                   36113 non-null  object 
 11  Q9_2                   36113 non-null  object 
 12  Q10_x                  36113 non-null  float64
 13  Q11_1               

,mb_sn,Q1,Q2,Q3,Q4,Q5,Q5_1,Q6,Q7,Q8,...,문항1_qpoll_join_250714,문항1_qpoll_join_250221,문항1_qpoll_join_250626,문항1_qpoll_join_250723,문항1_qpoll_join_250627,문항1_qpoll_join_250328,문항1_qpoll_join_250703,문항1_qpoll_join_250611,문항1_qpoll_join_250304,문항2_qpoll_join_250304
0,w131784019041454,2.0,3.0,5.0,1.0,5,13,4.0,4.0,"1,2,3,4,5,,7,8,9,,11,,,14,15,,17,18,,20,,22,23...",...,0.0,0.0,Unknown,0.0,0.0,0.0,Unknown,0.0,0.0,0.0
1,w10403443472424,2.0,1.0,3.0,2.0,4,1,8.0,8.0,"1,2,3,4,5,,7,,9,,11,12,,,15,16,,,19,20,,22,23,...",...,3.0,0.0,Unknown,0.0,0.0,0.0,Unknown,0.0,0.0,0.0
2,w181629889821619,0.0,0.0,0.0,0.0,Unknown,Unknown,0.0,0.0,Unknown,...,0.0,0.0,Unknown,0.0,0.0,0.0,Unknown,0.0,0.0,0.0
3,w223949436321039,2.0,2.0,4.0,1.0,8,21,5.0,5.0,"1,2,,4,5,6,,,9,,11,12,,,15,,,,,,,22,23,,25,,,28",...,0.0,1.0,Unknown,0.0,0.0,0.0,Unknown,0.0,3.0,4.0
4,w303750980346998,1.0,0.0,1.0,3.0,4,1,3.0,3.0,"1,2,,4,5,,,,9,,,,,,,,,18,,20,,22,23,,25,26,,",...,0.0,0.0,Unknown,0.0,0.0,0.0,Unknown,0.0,0.0,0.0



정제된 데이터가 '../data/processed/02_cleaned_data.csv'에 저장되었습니다.
저장된 파일 크기: (36113, 67)

🎉 데이터 정제 완료!
이제 분석을 진행할 수 있습니다.
